In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Reads data
features = pd.read_csv("data/features.csv")
target = pd.read_csv("data/target.csv")

In [38]:
# Bins target values
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=4, encode = "onehot-dense", strategy = "quantile")

target_discretized = discretizer.fit_transform(target["G3"].values.reshape(-1, 1))
target_new = []
for row in target_discretized:
    target_new.append(list(row).index(1))

In [3]:
# Splits data into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 1001)

In [8]:
# Gets final grade for target
t_train = y_train["G3"]
t_test = y_test["G3"]

In [24]:
# Imports Ridge and accuracy metrics
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs Ridge Regression
model = Ridge().fit(X=X_train, y=t_train)

# Measures accuracy
print("R-squared value for training set: ", r2_score(t_train, model.predict(X_train)))
print("R-squared value for testing set: ", r2_score(t_test, model.predict(X_test)))

R-squared value for training set:  0.34042660318538653
R-squared value for testing set:  0.22771777942292648


In [40]:
# Splits data for classification into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_new, random_state = 1001)

In [43]:
# Imports SVC and accuracy metrics
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs SVC
model = SVC(gamma = "scale").fit(X=X_train, y=y_train)

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 37.42%
Prediction accuracy on the test data: 33.72%


In [44]:
# Imports SVC and accuracy metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs SVC
model = DecisionTreeClassifier().fit(X=X_train, y=y_train)

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 100.00%
Prediction accuracy on the test data: 35.63%


In [5]:
import torch

class NeuralNetwork(torch.nn.Module):
    def __init__(self, inputDim, outputDim, layerSize):
        super(NeuralNetwork, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.layerSize = layerSize

        self.fc1

        
    def forward(self, x):
        output